In [121]:
from player import Player
from dealer import Dealer
from table import Table
from utils import card_value

In [122]:
#main loop to play one whole shoe
p1 = Player()
d1 = Dealer()

table = Table(d1, [p1], True)

    
    

In [123]:
table.play_shoe()

NEW ROUND
player bets[1]
dealer hand ['5', 'A']
player hands [['7', '7']]
player count = 1 
['7', '5']
[SPLIT] Hand 0 split into 2 hands total
[STAND] Hand 0 stands
[STAND] Hand 1 stands


2

In [124]:
hands = [1, 4, 5, 7]
idx = 0

while idx < len(hands):
    print(idx)
    if idx == 1:
        hands.append(8)
    idx += 1


0
1
2
3
4
